In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.compose import ColumnTransformer

In [2]:
cash_book_path = 'C:\\Users\\SQL\\Documents\\GL_CB_202311291624.csv'
cash_book = pd.read_csv(cash_book_path)
cash_book

,DOCKEY,DOCNO,DOCTYPE,DOCDATE,POSTDATE,TAXDATE,DESCRIPTION,DESCRIPTION2,PAYMENTMETHOD,AREA,...,LOCALDOCAMT,FROMDOCTYPE,BOUNCEDDATE,GLTRANSID,CANCELLED,DEPOSITKEY,UPDATECOUNT,PRINTCOUNT,ATTACHMENTS,NOTE
0,320,PV-1311/001,PV,2013-11-30,2013-11-30,2013-11-30,NG KOK LIANG,Payment For SSM Registration and Printing,301-001,----,...,3209.00,NaN,NaN,167,F,NaN,NaN,0,NaN,NaN
1,322,PV-1311/002,PV,2013-11-20,2013-11-20,2013-11-20,OLIVER LEE FU ONN,Payment For Nov'13 Staff Salary,301-001,----,...,3270.25,NaN,NaN,168,F,NaN,NaN,0,NaN,NaN
2,328,PV-1311/003,PV,2013-11-30,2013-11-30,2013-11-30,KUMPULAN WANG SIMPANAN PEKERJA,Payment For Nov'13 Staff EPF Contribution,301-001,----,...,480.00,NaN,NaN,169,F,NaN,NaN,0,NaN,NaN
3,330,PV-1311/004,PV,2013-11-30,2013-11-30,2013-11-30,PERTUBUHAN KESELAMATAN SOSIAL,Payment For Nov'13 Staff Socso,301-001,----,...,43.90,NaN,NaN,170,F,NaN,NaN,0,NaN,NaN
4,332,PV-1312/001,PV,2013-12-31,2013-12-31,2013-12-31,KUMPULAN WANG SIMPANAN PEKERJA,Payment For Dec'13 Staff EPF Contribution,301-001,----,...,480.00,NaN,NaN,171,F,NaN,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17801,78807,OR 220524,OR,2022-05-24,2022-05-24,2022-05-24,SHENZHEN STS MICROELECTRONICS CO LTD,Payment For Invoice - IN2201-021,301-003,CN,...,28277.80,PM,NaN,39129,F,NaN,NaN,0,NaN,NaN
17802,78809,OR 220525,OR,2022-05-26,2022-05-26,2022-05-26,PALMTOP VEGEOIL PRODUCTS SDN BHD,Payment For Invoice - IN2204-037,301-003,JB,...,295.00,PM,NaN,39167,F,NaN,NaN,0,NaN,NaN
17803,78811,OR 220526,OR,2022-05-27,2022-05-27,2022-05-27,FGV JOHOR BULKERS SDN BHD,Payment For Invoice - IN2204-124,301-003,JB,...,1060.00,PM,NaN,39196,F,NaN,NaN,0,NaN,NaN
17804,78813,OR 220527,OR,2022-05-27,2022-05-27,2022-05-27,ANALOG DEVICES SDN.BHD,Payment For Invoice - IN2204-052,301-003,----,...,2250.00,PM,NaN,39197,F,NaN,NaN,0,NaN,NaN


In [3]:
cash_book_detail_path = 'C:\\Users\\SQL\\Documents\\GL_CBDTL_202311291624.csv'
cash_book_detail = pd.read_csv(cash_book_detail_path)
cash_book_detail

C:\Users\SQL\AppData\Local\Temp\ipykernel_4000\3886522046.py:2: DtypeWarning: Columns (13,15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  cash_book_detail = pd.read_csv(cash_book_detail_path)


,DTLKEY,DOCKEY,SEQ,AREA,AGENT,PROJECT,CODE,DESCRIPTION,GST_DOCDATE,GST_DOCNO,...,TARIFF,TAXRATE,TAXAMT,LOCALTAXAMT,TAXINCLUSIVE,AMOUNT,LOCALAMOUNT,CURRENCYCODE,CURRENCYRATE,CURRENCYAMOUNT
0,321,320,1,----,----,----,711-000,Ng Kok Liang-SSM Registration & Documents Prin...,NaN,NaN,...,NaN,NaN,0.0,0.0,0,3209.00,3209.00,----,1.000000,3209.00
1,323,322,1,----,----,----,719-000,Oliver Lee Fu Onn-Nov'13 Salary,NaN,NaN,...,NaN,NaN,0.0,0.0,0,2000.00,2000.00,----,1.000000,2000.00
2,325,322,2,----,----,----,720-000,Oliver Lee Fu Onn-Nov'13 Allowance,NaN,NaN,...,NaN,NaN,0.0,0.0,0,1500.00,1500.00,----,1.000000,1500.00
3,326,322,3,----,----,----,708-000,Oliver Lee Fu Onn-Nov'13 Salary,NaN,NaN,...,NaN,NaN,0.0,0.0,0,-220.00,-220.00,----,1.000000,-220.00
4,327,322,4,----,----,----,721-000,Oliver Lee Fu Onn-Nov'13 Salary,NaN,NaN,...,NaN,NaN,0.0,0.0,0,-9.75,-9.75,----,1.000000,-9.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56561,78808,78807,1,CN,CADEN HENG,----,300-S0029,Payment For Invoice - IN2201-021,NaN,NaN,...,NaN,NaN,0.0,0.0,0,28277.80,28277.80,US$,4.286139,6597.50
56562,78810,78809,1,JB,ALAN TANG,----,300-P0022,Payment For Invoice - IN2204-037,NaN,NaN,...,NaN,NaN,0.0,0.0,0,295.00,295.00,----,1.000000,295.00
56563,78812,78811,1,JB,ALAN TANG,----,300-F0005,Payment For Invoice - IN2204-124,NaN,NaN,...,NaN,NaN,0.0,0.0,0,1060.00,1060.00,----,1.000000,1060.00
56564,78814,78813,1,----,KK KUOK,----,300-A0010,Payment For Invoice - IN2204-052,NaN,NaN,...,NaN,NaN,0.0,0.0,0,2250.00,2250.00,----,1.000000,2250.00


In [4]:
print(cash_book.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17806 entries, 0 to 17805
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   DOCKEY             17806 non-null  int64  
 1   DOCNO              17806 non-null  object 
 2   DOCTYPE            17806 non-null  object 
 3   DOCDATE            17806 non-null  object 
 4   POSTDATE           17806 non-null  object 
 5   TAXDATE            17806 non-null  object 
 6   DESCRIPTION        17806 non-null  object 
 7   DESCRIPTION2       8175 non-null   object 
 8   PAYMENTMETHOD      17806 non-null  object 
 9   AREA               17806 non-null  object 
 10  AGENT              17806 non-null  object 
 11  PROJECT            17806 non-null  object 
 12  JOURNAL            17806 non-null  object 
 13  CHEQUENUMBER       14104 non-null  object 
 14  CURRENCYCODE       17806 non-null  object 
 15  CURRENCYRATE       17806 non-null  float64
 16  BANKCHARGE         177

In [5]:
print(cash_book_detail.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56566 entries, 0 to 56565
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   DTLKEY          56566 non-null  int64  
 1   DOCKEY          56566 non-null  int64  
 2   SEQ             56566 non-null  int64  
 3   AREA            56566 non-null  object 
 4   AGENT           56566 non-null  object 
 5   PROJECT         56566 non-null  object 
 6   CODE            56566 non-null  object 
 7   DESCRIPTION     56564 non-null  object 
 8   GST_DOCDATE     27337 non-null  object 
 9   GST_DOCNO       26807 non-null  object 
 10  COMPANYNAME     27576 non-null  object 
 11  REGISTERNO      0 non-null      float64
 12  GSTNO           5593 non-null   object 
 13  PERMITNO        4 non-null      object 
 14  COUNTRY         0 non-null      float64
 15  TAX             6747 non-null   object 
 16  TARIFF          0 non-null      float64
 17  TAXRATE         5707 non-null  

In [6]:
cash_book_column = cash_book.columns
print(cash_book_column)

Index(['DOCKEY', 'DOCNO', 'DOCTYPE', 'DOCDATE', 'POSTDATE', 'TAXDATE',
       'DESCRIPTION', 'DESCRIPTION2', 'PAYMENTMETHOD', 'AREA', 'AGENT',
       'PROJECT', 'JOURNAL', 'CHEQUENUMBER', 'CURRENCYCODE', 'CURRENCYRATE',
       'BANKCHARGE', 'BANKCHARGEACCOUNT', 'DOCAMT', 'LOCALDOCAMT',
       'FROMDOCTYPE', 'BOUNCEDDATE', 'GLTRANSID', 'CANCELLED', 'DEPOSITKEY',
       'UPDATECOUNT', 'PRINTCOUNT', 'ATTACHMENTS', 'NOTE'],
      dtype='object')


In [7]:
cash_book_detail_column = cash_book_detail.columns
print(cash_book_detail_column)

Index(['DTLKEY', 'DOCKEY', 'SEQ', 'AREA', 'AGENT', 'PROJECT', 'CODE',
       'DESCRIPTION', 'GST_DOCDATE', 'GST_DOCNO', 'COMPANYNAME', 'REGISTERNO',
       'GSTNO', 'PERMITNO', 'COUNTRY', 'TAX', 'TARIFF', 'TAXRATE', 'TAXAMT',
       'LOCALTAXAMT', 'TAXINCLUSIVE', 'AMOUNT', 'LOCALAMOUNT', 'CURRENCYCODE',
       'CURRENCYRATE', 'CURRENCYAMOUNT'],
      dtype='object')


In [8]:
cash_book = cash_book.drop(['DOCNO', 'POSTDATE', 'TAXDATE', 'PAYMENTMETHOD', 'AREA', 'AGENT', 'PROJECT', 'JOURNAL', 'CHEQUENUMBER', 'CURRENCYCODE', 'CURRENCYRATE', 'BANKCHARGE', 'BANKCHARGEACCOUNT', 'DOCAMT', 'LOCALDOCAMT', 'FROMDOCTYPE', 'BOUNCEDDATE', 'GLTRANSID', 'CANCELLED', 'DEPOSITKEY', 'UPDATECOUNT', 'PRINTCOUNT', 'ATTACHMENTS', 'NOTE'], axis=1)
cash_book_detail = cash_book_detail.drop(['DTLKEY', 'SEQ', 'AREA', 'AGENT', 'PROJECT', 'GST_DOCDATE', 'GST_DOCNO', 'COMPANYNAME', 'REGISTERNO', 'GSTNO', 'PERMITNO', 'COUNTRY', 'TAX', 'TARIFF', 'TAXRATE', 'TAXAMT','LOCALTAXAMT', 'TAXINCLUSIVE', 'AMOUNT', 'LOCALAMOUNT', 'CURRENCYCODE', 'CURRENCYRATE'], axis=1)

In [9]:
merged_cash_book = pd.merge(cash_book, cash_book_detail, on='DOCKEY')

In [10]:
merged_cash_book

,DOCKEY,DOCTYPE,DOCDATE,DESCRIPTION_x,DESCRIPTION2,CODE,DESCRIPTION_y,CURRENCYAMOUNT
0,320,PV,2013-11-30,NG KOK LIANG,Payment For SSM Registration and Printing,711-000,Ng Kok Liang-SSM Registration & Documents Prin...,3209.00
1,322,PV,2013-11-20,OLIVER LEE FU ONN,Payment For Nov'13 Staff Salary,719-000,Oliver Lee Fu Onn-Nov'13 Salary,2000.00
2,322,PV,2013-11-20,OLIVER LEE FU ONN,Payment For Nov'13 Staff Salary,720-000,Oliver Lee Fu Onn-Nov'13 Allowance,1500.00
3,322,PV,2013-11-20,OLIVER LEE FU ONN,Payment For Nov'13 Staff Salary,708-000,Oliver Lee Fu Onn-Nov'13 Salary,-220.00
4,322,PV,2013-11-20,OLIVER LEE FU ONN,Payment For Nov'13 Staff Salary,721-000,Oliver Lee Fu Onn-Nov'13 Salary,-9.75
...,...,...,...,...,...,...,...,...
56561,78807,OR,2022-05-24,SHENZHEN STS MICROELECTRONICS CO LTD,Payment For Invoice - IN2201-021,300-S0029,Payment For Invoice - IN2201-021,6597.50
56562,78809,OR,2022-05-26,PALMTOP VEGEOIL PRODUCTS SDN BHD,Payment For Invoice - IN2204-037,300-P0022,Payment For Invoice - IN2204-037,295.00
56563,78811,OR,2022-05-27,FGV JOHOR BULKERS SDN BHD,Payment For Invoice - IN2204-124,300-F0005,Payment For Invoice - IN2204-124,1060.00
56564,78813,OR,2022-05-27,ANALOG DEVICES SDN.BHD,Payment For Invoice - IN2204-052,300-A0010,Payment For Invoice - IN2204-052,2250.00


In [11]:
# Convert 'DOCDATE' to datetime format
merged_cash_book['DOCDATE'] = pd.to_datetime(merged_cash_book['DOCDATE'], format='%Y-%m-%d')

# Extract day, month, and year
merged_cash_book['DAY'] = merged_cash_book['DOCDATE'].dt.day
merged_cash_book['MONTH'] = merged_cash_book['DOCDATE'].dt.month
merged_cash_book['YEAR'] = merged_cash_book['DOCDATE'].dt.year

# Display the modified DataFrame
print(merged_cash_book.head())

   DOCKEY DOCTYPE    DOCDATE      DESCRIPTION_x  \
0     320      PV 2013-11-30       NG KOK LIANG   
1     322      PV 2013-11-20  OLIVER LEE FU ONN   
2     322      PV 2013-11-20  OLIVER LEE FU ONN   
3     322      PV 2013-11-20  OLIVER LEE FU ONN   
4     322      PV 2013-11-20  OLIVER LEE FU ONN   

                                DESCRIPTION2     CODE  \
0  Payment For SSM Registration and Printing  711-000   
1            Payment For Nov'13 Staff Salary  719-000   
2            Payment For Nov'13 Staff Salary  720-000   
3            Payment For Nov'13 Staff Salary  708-000   
4            Payment For Nov'13 Staff Salary  721-000   

                                       DESCRIPTION_y  CURRENCYAMOUNT  DAY  \
0  Ng Kok Liang-SSM Registration & Documents Prin...         3209.00   30   
1                    Oliver Lee Fu Onn-Nov'13 Salary         2000.00   20   
2                 Oliver Lee Fu Onn-Nov'13 Allowance         1500.00   20   
3                    Oliver Lee Fu Onn-Nov

In [12]:
# Assuming you have a DataFrame named 'merged_cash_book'

# Define a dictionary with current column names as keys and new names as values
column_name_mapping = {
    'DOCKEY': 'DOCKEY',
    'DOCTYPE': 'DOCTYPE',
    'DOCDATE': 'DOCDATE',
    'DESCRIPTION_x': 'DESCRIPTION1',
    'DESCRIPTION2': 'DESCRIPTION2',
    'CODE': 'CODE',
    'DESCRIPTION_y': 'DESCRIPTION3',
    'CURRENCYAMOUNT': 'CURRENCYAMOUNT',
    'DAY': 'DAY',
    'MONTH': 'MONTH',
    'YEAR': 'YEAR',
}

# Rename the columns using the dictionary
merged_cash_book.rename(columns=column_name_mapping, inplace=True)

In [13]:
# Define the desired order of columns
desired_columns_order = ['DOCKEY', 'DOCTYPE', 'DOCDATE', 'DAY', 'MONTH', 'YEAR', 'CODE', 'DESCRIPTION1', 'DESCRIPTION2', 'DESCRIPTION3', 'CURRENCYAMOUNT']

# Reorder the columns in the DataFrame
merged_cash_book = merged_cash_book[desired_columns_order]

# Display the modified DataFrame
merged_cash_book

,DOCKEY,DOCTYPE,DOCDATE,DAY,MONTH,YEAR,CODE,DESCRIPTION1,DESCRIPTION2,DESCRIPTION3,CURRENCYAMOUNT
0,320,PV,2013-11-30,30,11,2013,711-000,NG KOK LIANG,Payment For SSM Registration and Printing,Ng Kok Liang-SSM Registration & Documents Prin...,3209.00
1,322,PV,2013-11-20,20,11,2013,719-000,OLIVER LEE FU ONN,Payment For Nov'13 Staff Salary,Oliver Lee Fu Onn-Nov'13 Salary,2000.00
2,322,PV,2013-11-20,20,11,2013,720-000,OLIVER LEE FU ONN,Payment For Nov'13 Staff Salary,Oliver Lee Fu Onn-Nov'13 Allowance,1500.00
3,322,PV,2013-11-20,20,11,2013,708-000,OLIVER LEE FU ONN,Payment For Nov'13 Staff Salary,Oliver Lee Fu Onn-Nov'13 Salary,-220.00
4,322,PV,2013-11-20,20,11,2013,721-000,OLIVER LEE FU ONN,Payment For Nov'13 Staff Salary,Oliver Lee Fu Onn-Nov'13 Salary,-9.75
...,...,...,...,...,...,...,...,...,...,...,...
56561,78807,OR,2022-05-24,24,5,2022,300-S0029,SHENZHEN STS MICROELECTRONICS CO LTD,Payment For Invoice - IN2201-021,Payment For Invoice - IN2201-021,6597.50
56562,78809,OR,2022-05-26,26,5,2022,300-P0022,PALMTOP VEGEOIL PRODUCTS SDN BHD,Payment For Invoice - IN2204-037,Payment For Invoice - IN2204-037,295.00
56563,78811,OR,2022-05-27,27,5,2022,300-F0005,FGV JOHOR BULKERS SDN BHD,Payment For Invoice - IN2204-124,Payment For Invoice - IN2204-124,1060.00
56564,78813,OR,2022-05-27,27,5,2022,300-A0010,ANALOG DEVICES SDN.BHD,Payment For Invoice - IN2204-052,Payment For Invoice - IN2204-052,2250.00


In [14]:
print(cash_book_detail.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56566 entries, 0 to 56565
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   DOCKEY          56566 non-null  int64  
 1   CODE            56566 non-null  object 
 2   DESCRIPTION     56564 non-null  object 
 3   CURRENCYAMOUNT  56566 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 1.7+ MB
None


In [15]:
# Specify the path where you want to save the CSV file
csv_file_path = 'DescriptionData.csv'

# Export the DataFrame to a CSV file
merged_cash_book.to_csv(csv_file_path, index=False)

In [16]:
description_path = 'C:\\Users\\SQL\\Documents\\DescriptionData.csv'
description = pd.read_csv(description_path)
description

,DOCKEY,DOCTYPE,DOCDATE,DAY,MONTH,YEAR,CODE,DESCRIPTION1,DESCRIPTION2,DESCRIPTION3,CURRENCYAMOUNT
0,320,PV,30/11/2013,30,11,2013,711-000,NG KOK LIANG,Payment For SSM Registration and Printing,Ng Kok Liang-SSM Registration & Documents Prin...,3209.00
1,322,PV,20/11/2013,20,11,2013,719-000,OLIVER LEE FU ONN,Payment For Nov'13 Staff Salary,Oliver Lee Fu Onn-Nov'13 Salary,2000.00
2,322,PV,20/11/2013,20,11,2013,720-000,OLIVER LEE FU ONN,Payment For Nov'13 Staff Salary,Oliver Lee Fu Onn-Nov'13 Allowance,1500.00
3,322,PV,20/11/2013,20,11,2013,708-000,OLIVER LEE FU ONN,Payment For Nov'13 Staff Salary,Oliver Lee Fu Onn-Nov'13 Salary,-220.00
4,322,PV,20/11/2013,20,11,2013,721-000,OLIVER LEE FU ONN,Payment For Nov'13 Staff Salary,Oliver Lee Fu Onn-Nov'13 Salary,-9.75
...,...,...,...,...,...,...,...,...,...,...,...
56561,78807,OR,24/5/2022,24,5,2022,300-S0029,SHENZHEN STS MICROELECTRONICS CO LTD,Payment For Invoice - IN2201-021,Payment For Invoice - IN2201-021,6597.50
56562,78809,OR,26/5/2022,26,5,2022,300-P0022,PALMTOP VEGEOIL PRODUCTS SDN BHD,Payment For Invoice - IN2204-037,Payment For Invoice - IN2204-037,295.00
56563,78811,OR,27/5/2022,27,5,2022,300-F0005,FGV JOHOR BULKERS SDN BHD,Payment For Invoice - IN2204-124,Payment For Invoice - IN2204-124,1060.00
56564,78813,OR,27/5/2022,27,5,2022,300-A0010,ANALOG DEVICES SDN.BHD,Payment For Invoice - IN2204-052,Payment For Invoice - IN2204-052,2250.00


In [17]:
description_column = description.columns
print(description_column)

Index(['DOCKEY', 'DOCTYPE', 'DOCDATE', 'DAY', 'MONTH', 'YEAR', 'CODE',
       'DESCRIPTION1', 'DESCRIPTION2', 'DESCRIPTION3', 'CURRENCYAMOUNT'],
      dtype='object')


In [18]:
description.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56566 entries, 0 to 56565
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   DOCKEY          56566 non-null  int64  
 1   DOCTYPE         56566 non-null  object 
 2   DOCDATE         56566 non-null  object 
 3   DAY             56566 non-null  int64  
 4   MONTH           56566 non-null  int64  
 5   YEAR            56566 non-null  int64  
 6   CODE            56566 non-null  object 
 7   DESCRIPTION1    56566 non-null  object 
 8   DESCRIPTION2    25284 non-null  object 
 9   DESCRIPTION3    56564 non-null  object 
 10  CURRENCYAMOUNT  56566 non-null  float64
dtypes: float64(1), int64(4), object(6)
memory usage: 4.7+ MB


In [19]:
# Split the dataset into training and testing sets
train_data, test_data = train_test_split(description, test_size=0.2, random_state=42)

# Define features and target
train_data = train_data.dropna()
X_train = train_data[['DOCTYPE', 'DAY', 'MONTH', 'YEAR', 'CODE', 'CURRENCYAMOUNT']]
y_train = train_data['DESCRIPTION3']

X_test = test_data[['DOCTYPE', 'DAY', 'MONTH', 'YEAR', 'CODE', 'CURRENCYAMOUNT']]
y_test = test_data['DESCRIPTION3']

print(X_train.shape)
print(y_train.shape)

(20316, 6)
(20316,)


In [20]:
# Define categorical columns
categorical_columns = ['DOCTYPE', 'CODE']

# One-hot encode categorical columns for training data
encoder = OneHotEncoder()
X_train_encoded = pd.get_dummies(X_train[categorical_columns])
columns_after_encoding = X_train_encoded.columns

# Transform the test data using the same encoder
X_test_encoded = pd.get_dummies(X_test[categorical_columns])

# Ensure columns in X_test_encoded match the columns used during training
missing_cols = set(columns_after_encoding) - set(X_test_encoded.columns)
for col in missing_cols:
    X_test_encoded[col] = 0  # Add missing columns with default value (0 or any other suitable value)

C:\Users\SQL\AppData\Local\Temp\ipykernel_4000\2860833159.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test_encoded[col] = 0  # Add missing columns with default value (0 or any other suitable value)
C:\Users\SQL\AppData\Local\Temp\ipykernel_4000\2860833159.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test_encoded[col] = 0  # Add missing columns with default value (0 or any other suitable value)
C:\Users\SQL\AppData\Local\Temp\ipykernel_4000\2860833159.py:15: PerformanceWarning: DataFrame is highly fragmented.

C:\Users\SQL\AppData\Local\Temp\ipykernel_4000\2860833159.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test_encoded[col] = 0  # Add missing columns with default value (0 or any other suitable value)
C:\Users\SQL\AppData\Local\Temp\ipykernel_4000\2860833159.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test_encoded[col] = 0  # Add missing columns with default value (0 or any other suitable value)
C:\Users\SQL\AppData\Local\Temp\ipykernel_4000\2860833159.py:15: PerformanceWarning: DataFrame is highly fragmented.

C:\Users\SQL\AppData\Local\Temp\ipykernel_4000\2860833159.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test_encoded[col] = 0  # Add missing columns with default value (0 or any other suitable value)
C:\Users\SQL\AppData\Local\Temp\ipykernel_4000\2860833159.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test_encoded[col] = 0  # Add missing columns with default value (0 or any other suitable value)
C:\Users\SQL\AppData\Local\Temp\ipykernel_4000\2860833159.py:15: PerformanceWarning: DataFrame is highly fragmented.

In [21]:
# Create the pipeline
pipeline = Pipeline([
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('encoder', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
        ],
        remainder='passthrough'
    )),
    ('classifier', RandomForestClassifier(max_depth=2, random_state=0))
])

In [22]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['DOCTYPE', 'CODE'])])),
                ('classifier',
                 RandomForestClassifier(max_depth=2, random_state=0))])

In [23]:
# Predictions on test data
y_pred = pipeline.predict(X_test)

In [24]:
# Evaluate the model on the test data
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print the results
print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

C:\Users\SQL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\SQL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.10650521477815096
Classification Report:
                                                                                                                                                                                                  precision    recall  f1-score   support

                                                                                                                                                             6 ROLL OF STRETCH FILM 1.8KG (20")       0.00      0.00      0.00         1
                                                                                                                                                   DEPOSIT REFUND - NO.8591-D,MALIM (PARTIALLY)       0.00      0.00      0.00         1
                                                                                                                                                                 MSL Hardware SB 94520-Material       0.00      0.00      0.00         1
             

C:\Users\SQL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
